In [3]:
from keras.layers import MaxPooling1D,Conv1D,UpSampling1D
from keras.layers import Dense, Input
from keras.models import Model
from keras import initializers
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import time


In [2]:
def bulid_CNNmodel(train_data, test_data):

    fnum1 = 16  ## filter1数目
    fnum2 = 1  ## filter2数目，如果最后一层是多卷积核，那么将出现多个平行降维数据，我们可以取其中一个，或者均值，或者直接设定一个卷积核
    ps1 = 25  ## pool_size1##这里的pooling要注意，因为存在整数运算逻辑，如果我们一维数据维度大小为1024，那么1024/25/10非正整数，将报错
    ps2 = 10  # #pool_size2##如果是2050，则降维后一维数据维度为2500/25/10=10
    input_msg = Input((train_data.shape[1], 1))
    encoded = Conv1D(filters=fnum1, kernel_size=4, strides=1, padding='same', activation='tanh',kernel_initializer=initializers.random_normal(stddev=0.01), bias_initializer='zeros')(input_msg)
    encoded = MaxPooling1D(pool_size=ps1)(encoded)
    encoded = Conv1D(filters=fnum2, kernel_size=6, strides=1, padding='same', activation='tanh',kernel_initializer=initializers.random_normal(stddev=0.01), bias_initializer='zeros')(encoded)
    encoder_output = MaxPooling1D(pool_size=ps2)(encoded)
    encoder_model = Model(inputs=input_msg, outputs=encoder_output)

    # 解码器，反过来
    decoded = UpSampling1D(size=ps2)(encoder_output)##上采样过程(反卷积)
    decoded = Conv1D(filters=fnum1, kernel_size=6, strides=1, padding='same', activation='tanh',kernel_initializer=initializers.random_normal(stddev=0.01), bias_initializer='zeros')(decoded)
    decoded = UpSampling1D(size=ps1)(decoded)
    decoded = Conv1D(filters=fnum2, kernel_size=4, strides=1, padding='same', activation='tanh',kernel_initializer=initializers.random_normal(stddev=0.01), bias_initializer='zeros')(decoded)

    # 转成原始数据大小
    decoded_output = Conv1D(filters=1, kernel_size=2, strides=1, padding='same', activation='tanh',kernel_initializer=initializers.random_normal(stddev=0.01), bias_initializer='zeros')(decoded)
    autoencoder = Model(input_msg, decoded_output)
    print(autoencoder.summary())  ## 打印网络结构
    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    print(train_data.shape, input_msg.shape, decoded_output.shape, encoder_output.shape)
    bs = int(len(train_data) / 4)  ##### 数据集较少，全参与形式，epochs一般跟batch_size成正比
    epochs = max(int(bs / 2), 128 * 3)
    a = autoencoder.fit(train_data, train_data, epochs=epochs, batch_size=bs, verbose=0, validation_split=0.2)## 在训练集中划分0.2作为测试集
    print('训练集Loss列表(长度%s)%s:' % (len(a.history['loss']), a.history['loss']))
    print('测试集Loss列表(长度%s)%s:' % (len(a.history['val_loss']), a.history['val_loss']))
    print('\033[1;31m{0:*^80}\033[0m'.format('测试集损失函数情况'))
    print(autoencoder.evaluate(test_data, test_data))  ## 观察测试集损失情况
    # encoder_model.save('临时保存的CNN模型.hdf5')
    return encoder_model,bs,a.history['loss'],a.history['val_loss'],autoencoder

In [ ]:



model = bulid_CNNmodel(traindata,testdata)


print()

In [ ]:
###预测和作图
def outputresult(data,model):
    modelres = model(data,data)
    dim_msg = modelres[0].predict(data)##此时预测是拿纯测试集预测
    print('降维后数据维度：',dim_msg.shape)
    dim_msg = np.reshape(dim_msg,(dim_msg.shape[0],dim_msg.shape[1]))
    latent_feature = pd.DataFrame(dim_msg)
    latent_feature.index = originaldata.index##read_res[2]
    latent_feature.columns = [('feature'+str(i + 1)) for i in range(dim_msg.shape[1])]
    latent_feature = np.round(latent_feature,6)
    print(latent_feature)

    plt.figure(figsize=(15, 8))
    plt.plot(modelres[2], label='训练集损失',)
    plt.plot(modelres[3], label='测试集损失')
    plt.xlabel('循环次数',fontsize=18)
    plt.tick_params(labelsize=18)
    plt.legend(fontsize=15)
    plt.show()
    return x_all_normalize.shape
outputresult(x_all_normalize,bulid_BPNNmodel)